### <b><span style='color:#F1C40F'>|</span> Home task</b>

- Choose any store from the initial dataset
- Check the presence of nans and fill them
- Make a forecast for 30, 180, 270, 365 days ahead
- Perform model evaluation

In [2]:
import pandas as pd
import numpy as np

### Get data

In [3]:
def preprocess_data(df:pd.DataFrame) -> pd.DataFrame:
    df.date = pd.to_datetime(df.date)
    df['day_of_week'] = df['date'].dt.day_name()
    return df

In [4]:
# Not uploaded to github because it's too big
train_df = pd.read_csv('train.csv')
train_df = preprocess_data(train_df)

train_df

,id,date,store_nbr,family,sales,onpromotion,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Tuesday
1,1,2013-01-01,1,BABY CARE,0.000,0,Tuesday
2,2,2013-01-01,1,BEAUTY,0.000,0,Tuesday
3,3,2013-01-01,1,BEVERAGES,0.000,0,Tuesday
4,4,2013-01-01,1,BOOKS,0.000,0,Tuesday
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Tuesday
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Tuesday
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Tuesday
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Tuesday


In [5]:
event_df = pd.read_csv('holidays_events.csv')
event_df = preprocess_data(event_df)

event_df

,date,type,locale,locale_name,description,transferred,day_of_week
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False,Friday
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False,Sunday
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False,Thursday
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False,Saturday
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False,Saturday
...,...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False,Friday
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False,Saturday
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False,Sunday
348,2017-12-25,Holiday,National,Ecuador,Navidad,False,Monday


### Choice the store

In [6]:
store_nbr = 7
train_df = train_df.set_index('id')
train_df = train_df[train_df['store_nbr'] == store_nbr]


### Sum up sales for the day

In [7]:
def sum_sales_per_day(df, store_number):
    day_level_df = df[df["store_nbr"]==store_number]\
        [
            ["date", "sales", "day_of_week"]
        ]\
            .groupby("date").agg(
                {
                    "sales": "sum",
                    "day_of_week": "first"
                }).reset_index()

    return day_level_df

In [8]:
day_level_df = sum_sales_per_day(train_df, store_nbr)
day_level_df

,date,sales,day_of_week
0,2013-01-01,0.000000,Tuesday
1,2013-01-02,11997.502000,Wednesday
2,2013-01-03,9277.958000,Thursday
3,2013-01-04,8096.573018,Friday
4,2013-01-05,8610.605999,Saturday
...,...,...,...
1679,2017-08-11,15090.052970,Friday
1680,2017-08-12,14668.772022,Saturday
1681,2017-08-13,14871.927000,Sunday
1682,2017-08-14,19957.600000,Monday


### Visualisation

In [9]:
import plotly.express as px

fig = px.line(day_level_df, x='date', y=["sales"], markers=True, title="Store sales")
fig.show()

In [10]:
fig = px.box(day_level_df, x='day_of_week', y="sales", color="day_of_week",
             boxmode="overlay", points='all')
fig.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    width=1400,
    height=700,
    title='Weekdays sales distribution',
)

### Check the presence of nans and fill them

In [11]:
day_level_df["sales"] = day_level_df["sales"].mask(day_level_df["sales"] == float(0), None)
day_level_df.fillna(0, inplace=True)

# Make a forecast for 30, 180, 270, 365 days ahead

In [12]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

### Preprocess data to needed format

In [13]:
fbp_set = day_level_df[['date', 'sales']]
fbp_set.rename(columns={"date": "ds", "sales":"y"}, inplace=True)
fbp_set.fillna(0, inplace=True)
fbp_set

C:\Users\Admin\AppData\Local\Temp\ipykernel_18876\1022911304.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Admin\AppData\Local\Temp\ipykernel_18876\1022911304.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,y
0,2013-01-01,0.000000
1,2013-01-02,11997.502000
2,2013-01-03,9277.958000
3,2013-01-04,8096.573018
4,2013-01-05,8610.605999
...,...,...
1679,2017-08-11,15090.052970
1680,2017-08-12,14668.772022
1681,2017-08-13,14871.927000
1682,2017-08-14,19957.600000


In [14]:
holiday_df = event_df.copy()
holiday_df.rename(columns={"date": "ds", "description":"holiday"}, inplace=True)
holiday_df = holiday_df[["ds", "holiday"]]
holiday_df.head()

,ds,holiday
0,2012-03-02,Fundacion de Manta
1,2012-04-01,Provincializacion de Cotopaxi
2,2012-04-12,Fundacion de Cuenca
3,2012-04-14,Cantonizacion de Libertad
4,2012-04-21,Cantonizacion de Riobamba


### Functions for forecast

In [15]:
def evaluate_forecasting_model(actual_values, predicted_values, round_nbr):
    mape = mean_absolute_percentage_error(
        actual_values,
        predicted_values
    )
    mae = mean_absolute_error(
        actual_values,
        predicted_values
    )
    mse = mean_squared_error(
        actual_values,      
        predicted_values
    )
    
    print(f"MAE - {round(mae, round_nbr)}")
    print(f"MSE - {round(mse, round_nbr)}")
    print(f"MAPE - {round(mape, round_nbr)}")
    
def evaluate_prophet(df, holiday_df, window, round_nbr):
    train, test = df[:-window], df[-window:]
    
    m = Prophet(holidays=holiday_df)
    m.fit(train)
    
    future_sales = m.make_future_dataframe(periods=window)
    forecast = m.predict(future_sales).iloc[-365:]
    
    evaluate_forecasting_model(test[:window]['y'], forecast[:window]['yhat'], round_nbr)

### 30 days

In [16]:
print('Forecast for 30 days:')
evaluate_prophet(fbp_set, holiday_df, 30, store_nbr)

Forecast for 30 days:


13:28:01 - cmdstanpy - INFO - Chain [1] start processing
13:28:03 - cmdstanpy - INFO - Chain [1] done processing


MAE - 2442.6875169
MSE - 9799635.9996374
MAPE - 0.1448133


### 180 days

In [17]:
print('Forecast for 180 days:')
evaluate_prophet(fbp_set, holiday_df, 180, store_nbr)

Forecast for 180 days:


13:28:05 - cmdstanpy - INFO - Chain [1] start processing
13:28:06 - cmdstanpy - INFO - Chain [1] done processing


MAE - 3081.0875663
MSE - 15695946.621189
MAPE - 0.1550964


### 270 days

In [18]:
print('Forecast for 270 days:')
evaluate_prophet(fbp_set, holiday_df, 270, store_nbr)

Forecast for 270 days:


13:28:08 - cmdstanpy - INFO - Chain [1] start processing
13:28:08 - cmdstanpy - INFO - Chain [1] done processing


MAE - 2680.854649
MSE - 13358301.5723166
MAPE - 3.5545742234762854e+17


### 365 days

In [19]:
print('Forecast for 365 days:')
evaluate_prophet(fbp_set, holiday_df, 365, store_nbr)

Forecast for 365 days:


13:28:10 - cmdstanpy - INFO - Chain [1] start processing
13:28:12 - cmdstanpy - INFO - Chain [1] done processing


MAE - 2604.1316811
MSE - 12233660.5996993
MAPE - 2.4358933156337984e+17
